# 知识工程-作业8 中文事件抽取
2024214500 叶璨铭


## 代码与文档格式说明

> 本文档使用Jupyter Notebook编写，遵循Diátaxis 系统 Notebook实践 https://nbdev.fast.ai/tutorials/best_practices.html，所以同时包括了实验文档和实验代码。

> 本文档理论上支持多个格式，包括ipynb, docx, pdf 等。您在阅读本文档时，可以选择您喜欢的格式来进行阅读，建议您使用 Visual Studio Code (或者其他支持jupyter notebook的IDE, 但是VSCode阅读体验最佳) 打开 `ipynb`格式的文档来进行阅读。

> 为了记录我们自己修改了哪些地方，使用git进行版本控制，这样可以清晰地看出我们基于助教的代码在哪些位置进行了修改，有些修改是实现了要求的作业功能，而有些代码是对原本代码进行了重构和优化。我将我在知识工程课程的代码，在作业截止DDL之后，开源到 https://github.com/2catycm/THU-Coursework-Knowledge-Engineering.git ，方便各位同学一起学习讨论。


## 代码规范说明

在我们实现函数过程中，函数的docstring应当遵循fastai规范而不是numpy规范，这样简洁清晰，不会Repeat yourself。相应的哲学和具体区别可以看 
https://nbdev.fast.ai/tutorials/best_practices.html#keep-docstrings-short-elaborate-in-separate-cells


为了让代码清晰规范，在作业开始前，使用 `ruff format`格式化助教老师给的代码; 

![alt text](image.png)


哇！我们当场就检查出了代码错误！不只是格式化问题了，看看 metrics/m2scorer/Tokenizer.py:177:15 和 metrics/m2scorer/token_offsets.py:43:15 是怎么回事

![alt text](image-2.png)

![alt text](image-1.png)


原来是 m2scorer 太老了，居然用了 Python2 的语法！我们简单修改为 `print("")` 语法就可以了。
不过语句有点多啊，我们一个个改有点不够优雅。

Python官方有工具，
```bash
2to3 -w .
```

![alt text](image-3.png)

改了特别多东西

![alt text](image-4.png)

终于勉强看起来正常了，不过细看代码还是有很多不正常的地方，看来学长只是想让我们参考代码，这个应该是难以跑通的。



同时注意到VSCode-Pylance插件的报错



## 实验环境准备


上次作业结束的时候，我们注意到我们想要尝试的最新方法只能支持3.12，PyTorch2.4也不够新，所以我们这次作业重新创建一个作业专属3.12环境。

先装小依赖包, 然后安装最新pytorch

```bash
conda create -n assignments python=3.12
conda activate assignments
pip install -r ../requirements.txt
pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu126
```

注意到

```python
from elmoformanylangs import Embedder, logger
import einops
import jieba
```

参考 https://github.com/HIT-SCIR/ELMoForManyLangs， 安装

```bash
git submodule add https://github.com/HIT-SCIR/ELMoForManyLangs
cd ELMoForManyLangs
pip install -e .
```
PyTorch版本没有冲突，不用被乱装一通，太棒了。

安装一下上次没有探究完的 RWKV 
```bash
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```


## 原理回顾和课件复习



课上详细介绍了语法纠正任务的一些基本特点和难点，先介绍了规则模型和分类模型，然后开始介绍翻译任务用于语法纠正和噪声信道模型，然后详细介绍了 Seq2seq 的发展历程，最后提及了一下大模型方法。


## 数据准备

download.sh 的清华网盘链接过期了，还好学长给的压缩包已经处理好了数据 NLPCC 2018 Task 2，已经有“./data/processed/”

```python
train_dataset = GECDataset(
    "./data/processed/seg.train", vocab_dict=vocab_dict, max_length=200
)
test_dataset = GECDataset(
    "./data/processed/seg.txt", vocab_dict=vocab_dict, max_length=200
)
```



## 预训练模型下载

根据main.py, 首先我们需要下载  "zhs.model"
```python
elmo_model = Embedder("zhs.model", batch_size=16)
vocab_dict = load_vocab_dict("./zhs.model/word.dic")
```

根据 https://github.com/HIT-SCIR/ELMoForManyLangs
有两个链接都是下载中文模型的，特别强调后面的是简体中文。
```bash
wget http://vectors.nlpl.eu/repository/11/179.zip
wget http://39.96.43.154/zhs.model.tar.bz2
```
后者链接失效了无法连接上！

只好下载前面这个的 179.zip, 然后把里面的内容移动进去

```bash
unzip 179.zip
md zhs.model
mv char.dic config.json encoder.pkl meta.json README token_embedder.pkl word.dic zhs.model
rm 179.zip
```

## 评价指标 Max Match 实现

Max Match （因为有两个M 又叫作 M2 ）是由 NUS 的研究者在这篇2012 ACL 论文 https://aclanthology.org/N12-1067.pdf “Better Evaluation for Grammatical Error Correction” 提出的。

老师的课件指出这是应用最广泛的指标之一。

![alt text](image-5.png)


MaxMatch (M²) 首先第一个思路是不再直接比较生成句子和参考句子的相似度，而是比较系统所做的“编辑操作” (edits) 与人类标注者提供的“标准编辑操作”的匹配程度，也就是课件中说的ei和gi。

核心思想是“Max”，可以假设我们有多个标准答案，对于每一个人工标注的参考答案（即一个正确的句子版本），通过比较该参考答案和原始错误句子，找出从错误句子到这个特定正确版本所需的标准编辑操作。对于系统生成的句子所对应的一组编辑操作，M² 会尝试将其与每一个参考答案所对应的标准编辑操作集进行比较。

它会计算系统编辑集与每一个标准编辑集之间的匹配程度（通常是计算重叠的编辑数量）。
然后，它会选择那个能与系统编辑集产生最大匹配度（即最多重叠编辑）的参考答案。
重要的是， M² 认为，只要系统做出的编辑与 任意一个 正确的参考答案中的编辑相匹配，这个编辑就是有效的。这就是“MaxMatch”的含义——在所有可能的标准答案中，找到对系统最有利（匹配度最高）的那一个来进行评估。

M² 的主要目的是解决GEC评估中的模糊性问题：同一个错误修正问题可能有多种编辑操作序列导致相同的结果句子，而不同系统可能通过不同的编辑路径达到相同输出。


那么具体要怎么实现呢？经过我的初步调查，助教建议的第一个参考代码 https://github.com/shibing624/pycorrector 虽然看起来集成了很多方法，但是 max match 指标似乎没有（或者不叫这个名字），评测的代码是 “https://github.com/shibing624/pycorrector/blob/master/pycorrector/utils/evaluate_utils.py” 但是我还没有看懂，这里面引入了"SIGHAN", "句级评估结果", "设定需要纠错为正样本，无需纠错为负样本" 这些概念，好像用了对比学习，但是没有直接说是用 Max Match。

助教给我们的第二个代码就是 M² 的官方代码，也就是刚才我费了半天劲升级为 Python3 的代码。根据助教给的提示，我们可以直接外部调用其功能用来评估。


In [ ]:
from typing import *
import subprocess


def maxmatch_metric(prediction_file: str # a file containing predicted output
                    , label_file: str # a file containig groundtruth output
                    ) -> Any:
    """
    calculate maxmatch metrics

    File content example
    # prediction file
    ```
    冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    ```
    # label_file
    ```
    S 冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    A 9 11|||W|||虽然 它|||REQUIRED|||-NONE-|||0

    S 首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    A 17 18|||S|||榨|||REQUIRED|||-NONE-|||0
    A 38 39|||S|||六|||REQUIRED|||-NONE-|||0
    A 43 44|||S|||四 量杯|||REQUIRED|||-NONE-|||0
    A 49 50|||S|||十|||REQUIRED|||-NONE-|||0
    ```
    """
    subprocess.check_call(
        ["python", "metrics/m2scorer/m2scorer.py", prediction_file, label_file]
    )


为此，我特意把助教注释里面的文件内容拿了出来进行测试，看看是否评测正确。

![alt text](image-6.png)

确实得是0， prediction_file 没有改错，这个是啥都不干的语法修正器。

然后我按照grond truth的要求，让LLM遵循ground truth的命令更正语法，得到 prediction_file2

![alt text](image-7.png)

没想到居然不是完全对，precision差一个，意思是我们不小心多改了一个什么东西。

仔细检查了半天，发现原来是助教给我们的Python 注释有问题，冬阴功分词给合起来了，我们修改一下 Python 注释以及 predicition_file_correct.txt 就行。

![alt text](image-8.png)

我们还可以写正则表达式提取一下这三个数值出来，这样返回的时候更好处理

![alt text](image-9.png)

In [ ]:
def maxmatch_metric(prediction_file: str # a file containing predicted output
                    , label_file: str # a file containig groundtruth output
                    , verbose:bool = True
                    ) -> Any:
    """
    calculate maxmatch metrics

    File content example
    # prediction file
    ```
    冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    ```
    # label_file
    ```
    S 冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    A 9 11|||W|||虽然 它|||REQUIRED|||-NONE-|||0

    S 首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    A 17 18|||S|||榨|||REQUIRED|||-NONE-|||0
    A 38 39|||S|||六|||REQUIRED|||-NONE-|||0
    A 43 44|||S|||四 量杯|||REQUIRED|||-NONE-|||0
    A 49 50|||S|||十|||REQUIRED|||-NONE-|||0
    ```
    """
    # subprocess.check_call(
    #     ["python", "metrics/m2scorer/m2scorer.py", prediction_file, label_file]
    # )
    # 执行命令并捕获输出
    result = subprocess.check_output(
        ["python", "metrics/m2scorer/m2scorer.py", prediction_file, label_file],
        text=True  # 直接获取文本输出，无需解码
    )

    # 打印原始输出
    if verbose:
        print("m2scorer评测中:")
        print(result)

    # 使用正则表达式提取指标
    metrics = {}
    metrics_pattern = re.compile(r'(Precision|Recall|F0\.5)\s*:\s*([\d\.]+)')
    matches = metrics_pattern.findall(result)
    
    # 将匹配的结果转换为字典
    for key, value in matches:
        metrics[key] = float(value)

    return metrics

当然这样会被底层代码限制住，只能获得4位小数，虽然对我们这次实验够用了，但是为了避免后人再被这个m2scorer坑住，我们决定把里面的代码改造一些出来。

In [ ]:
from pathlib import Path
this_file = Path(__file__).resolve()
this_directory = this_file.parent
import sys
sys.path.append((this_directory/"m2scorer").as_posix())

import levenshtein as levenshtein
from util import paragraphs
from util import smart_open
from typing import Any

def load_annotation(gold_file):
    source_sentences = []
    gold_edits = []
    fgold = smart_open(gold_file, "r")
    puffer = fgold.read()
    fgold.close()
    # puffer = puffer.decode('utf8')
    for item in paragraphs(puffer.splitlines(True)):
        item = item.splitlines(False)
        sentence = [line[2:].strip() for line in item if line.startswith("S ")]
        assert sentence != []
        annotations = {}
        for line in item[1:]:
            if line.startswith("I ") or line.startswith("S "):
                continue
            assert line.startswith("A ")
            line = line[2:]
            fields = line.split("|||")
            start_offset = int(fields[0].split()[0])
            end_offset = int(fields[0].split()[1])
            etype = fields[1]
            if etype == "noop":
                start_offset = -1
                end_offset = -1
            corrections = [
                c.strip() if c != "-NONE-" else "" for c in fields[2].split("||")
            ]
            # NOTE: start and end are *token* offsets
            original = " ".join(" ".join(sentence).split()[start_offset:end_offset])
            annotator = int(fields[5])
            if annotator not in list(annotations.keys()):
                annotations[annotator] = []
            annotations[annotator].append(
                (start_offset, end_offset, original, corrections)
            )
        tok_offset = 0
        for this_sentence in sentence:
            tok_offset += len(this_sentence.split())
            source_sentences.append(this_sentence)
            this_edits = {}
            for annotator, annotation in annotations.items():
                this_edits[annotator] = [
                    edit
                    for edit in annotation
                    if edit[0] <= tok_offset
                    and edit[1] <= tok_offset
                    and edit[0] >= 0
                    and edit[1] >= 0
                ]
            if len(this_edits) == 0:
                this_edits[0] = []
            gold_edits.append(this_edits)
    return (source_sentences, gold_edits)


def maxmatch_metric(prediction_file: str # a file containing predicted output
                    , label_file: str # a file containig groundtruth output
                    , verbose:bool = True
                    ) -> Any:
    """
    calculate maxmatch metrics

    File content example
    # prediction file
    ```
    冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    ```
    # label_file
    ```
    S 冬 阴功 是 泰国 最 著名 的 菜 之一 ， 它 虽然 不 是 很 豪华 ， 但 它 的 味 确实 让 人 上瘾 ， 做法 也 不 难 、 不 复杂 。
    A 9 11|||W|||虽然 它|||REQUIRED|||-NONE-|||0

    S 首先 ， 我们 得 准备 : 大 虾六 到 九 只 、 盐 一 茶匙 、 已 搾 好 的 柠檬汁 三 汤匙 、 泰国 柠檬 叶三叶 、 柠檬 香草 一 根 、 鱼酱 两 汤匙 、 辣椒 6 粒 ， 纯净 水 4量杯 、 香菜 半量杯 和 草菇 10 个 。
    A 17 18|||S|||榨|||REQUIRED|||-NONE-|||0
    A 38 39|||S|||六|||REQUIRED|||-NONE-|||0
    A 43 44|||S|||四 量杯|||REQUIRED|||-NONE-|||0
    A 49 50|||S|||十|||REQUIRED|||-NONE-|||0
    ```
    """
    max_unchanged_words = 2
    beta = 0.5
    ignore_whitespace_casing = False
    very_verbose = False

    # load source sentences and gold edits
    source_sentences, gold_edits = load_annotation(label_file)

    # load system hypotheses
    fin = smart_open(prediction_file, "r")
    system_sentences = [line.strip() for line in fin.readlines()]
    fin.close()

    p, r, f1 = levenshtein.batch_multi_pre_rec_f1(
        system_sentences,
        source_sentences,
        gold_edits,
        max_unchanged_words,
        beta,
        ignore_whitespace_casing,
        verbose,
        very_verbose,
    )

    metrics = {
        "Precision": p,
        "Recall": r,
        "F_{}".format(beta): f1
    }

    return metrics

# 如果需要测试函数，可以调用它并打印结果
if __name__ == "__main__":
    prediction_file = (this_directory/"../data/test_prediction_file_correct.txt").as_posix()
    label_file = (this_directory/"../data/test_label_file.txt").as_posix()
    metrics = maxmatch_metric(prediction_file, label_file)
    print("Metrics:", metrics)


![alt text](image-10.png)

看来实现是正确的，而且我们还能看到详细的信息。

## 数据加载

我们看下 util.py 文件

助教这一次已经帮我们完美实现了 GECDataset 类，除了有些类型标注不严谨的问题，这里我们不修改已有的成熟代码，因为可以跑通。


## Encoder-Decoder 实现

我们首先复习一下课件

![alt text](image-11.png)

![alt text](image-12.png)


注意似乎和我更熟悉的 Transformer的 Encoder-Decoder 结构有点不一样。
这里是encoder得到单个 context vector, 然后 decoder 直接用这个 context vector 来进行预测。

而在Transformer模型中，Encoder通过交叉注意力机制（Cross-Attention）​将信息传递给Decoder。

![alt text](image-13.png)

Encoder得到了 N个D维输出（token数量），这些输出被Decoder的Cross Attention层使用，会产生 Key 和 Value 被使用。 Transformer论文这部分讲得不清不楚。

实际上每一个decoder层，都会先自己masked attention，然后再和encoder的结果去cross。


我们回到课件上讲的 传统 RNN 常用的 Encoder-Decoder，没有很多层。



### elmo 怎么用？
我们直接开始写，首先类型注释缺乏 elmo_model 的 类型，我们打开 https://github.com/
HIT-SCIR/ELMoForManyLangs 标注为 Embedder, 因为 main 里面是
```python
elmo_model = Embedder("zhs.model", batch_size=16)

```

随后有个重要的问题，elmo的embed的维度是多少？我们看了  7.中文语法错误纠正的作业/zhs.model/config.json 引用的 7.中文语法错误纠正的作业/ELMoForManyLangs/elmoformanylangs/configs/cnn_50_100_512_4096_sample.json 才知道是 512， 但是实际上有掉进坑里面了，实际上是1024

ELMoForManyLangs 我们查看那源代码发现是不太方便学习的，是个 object，我们需要自己同步tensor 的 device。


### GRU 怎么用？
然后的问题是 GRU， https://pytorch.org/docs/stable/generated/torch.nn.GRU.html

我搞懂了多层 GRU 原来是从下层的ht作为xt往上传，这里没有额外的产生网络。


现在我们来特别学习这个函数，
https://pytorch.org/docs/stable/generated/torch.nn.utils.rnn.pad_packed_sequence.html， 
它的作用是将填充后的序列打包成一个 PackedSequence 对象。这样做的好处是可以在 RNN 模型（例如 LSTM 或 GRU）中，只对有效的时间步进行计算，从而提高效率并避免填充部分对模型计算的干扰。

source_mask 是一个二值化的张量，用于指示每个序列中的有效位置（例如，1 表示有效，0 表示填充）。通过在维度1上求和，得到每个序列有效元素的个数，即序列真实的长度。


In [ ]:
from elmoformanylangs import Embedder
def encode(
        self,
        source_inputs: List[List[str]],  # a list of input text
        source_mask: torch.Tensor,  # size (batch_size, sequence_length)
        **kwargs,
    ) -> (
        torch.Tensor
    ):  # encoder_outputs, size (batch_size, sequence_length, hidden_states)
        """
        Encode input source text, using source_mask to pack padded sequences.
        """
        # Encode the source inputs using ELMo
        device = next(self.parameters()).device
        with torch.no_grad():
            elmo_outputs = self.elmo.sents2elmo(source_inputs)
            elmo_outputs = torch.FloatTensor(elmo_outputs).to(device)
        # Compute lengths from source_mask (assumes mask with 1 for valid tokens)
        lengths = source_mask.sum(dim=1)
        # Pack the padded sequence using the computed lengths
        packed_input = pack_padded_sequence(
            elmo_outputs, lengths.cpu(), batch_first=True, enforce_sorted=False
        )
        # Apply the encoder GRU
        packed_outputs, _ = self.encoder_gru(packed_input)
        # Unpack the sequence
        encoder_outputs, _ = pad_packed_sequence(packed_outputs, batch_first=True)
        return encoder_outputs

### Seq2Seq Attention

除了老师课件，我们还阅读了 https://pytorch.org/tutorials/intermediate/seq2seq_translation_tutorial.html

与 https://github.com/PatrickSVM/Seq2Seq-with-Attention/blob/main/seq2seq_attention/model.py

我们实际上使用的Attention是 nn.MultiheadAttention, 而不是课上学习的 Additive Attention。


## Beam Search 实现

在训练阶段，可以直接用ground truth的token 作为上一个时刻预测出来的token，这就是所谓的 teacher forcing。刚才代码里面是在 target_input_ids, target_inputs, target_mask 指定的。

推理阶段，局部top 1 贪心可能不是最优的，可能累计误差，所以需要 beam search。

beam search 保留每个时刻的top k个单词，然后下一个时刻使用这K个可能生成 K*L 个概率，其中选择 top K的作为下一个时刻的输出。

当我们实际写代码的时候，发现这个说法稍微有些不准确。实际上不是单词的概率，而是累计到那个位置序列的概率。


在开始写代码前要搞懂输入输出类型。


| **文件**   | **调用位置** | **方法** | **传入参数** | **参数类型** | **参数含义** |
|------------|--------------|----------|--------------|--------------|--------------|
| `main.py`  | 模型初始化   | `GECModel` 构造函数 | `elmo_model`<br>`len(vocab_dict)` | `Embedder`<br>`int` | ELMo模型实例<br>词汇表大小 |
| `main.py`  | 训练循环     | `model(**batch)` | `batch` | `Dict[str, Tensor]` | 包含输入数据的字典，键为 `"source_input_ids"`, `"source_mask"`, `"target_input_ids"` 等 |
| `main.py`  | 测试循环     | `generator.generate(**batch)` | `batch` | `Dict[str, Tensor]` | 包含输入数据的字典，键为 `"source_input_ids"`, `"source_mask"` 等 |
| `main.py`  | 模型初始化   | `BeamSearchGenerator` 构造函数 | `model`<br>`reverse_vocab_dict`<br>`device` | `GECModel`<br>`Dict[int, str]`<br>`str` | GEC模型实例<br>反向词汇表<br>设备（如 `"cuda:0"` 或 `"cpu"`） |


  - `batch`：类型为 `Dict[str, Tensor]`，是一个字典，包含以下键：
    - `"source_input_ids"`：源文本的输入ID，形状为 `(batch_size, seq_length)`。
    - `"source_mask"`：源文本的掩码，形状为 `(batch_size, seq_length)`。
    - `"target_input_ids"`：目标文本的输入ID，形状为 `(batch_size, seq_length)`。
    - `"target_mask"`：目标文本的掩码，形状为 `(batch_size, seq_length)`。
    - `"labels"`：目标文本的标签，形状为 `(batch_size, seq_length)`。



In [ ]:
class BeamSearchGenerator:
    def __init__(self, model: GECModel, # model.py 定义的模型
                 reverse_vocab_dict:Dict[int, str], # elmo 词典
                 device:str):
        self.oov_index = 0
        self.bos_index = 1
        self.eos_index = 2
        self.pad_index = 3
        self.model = model
        self.max_length = 200
        self.num_beams = 8
        self.length_penalty = 0.7
        self.vocab = reverse_vocab_dict
        self.vocab_size = len(reverse_vocab_dict) # 也就是 L 的大小。
        self.device = device

    def generate(self, source_mask, 
                 **kwargs):
        encoder_outputs = self.model.encode(**kwargs)
        generated_sequence = self.beam_search(encoder_outputs, source_mask)
        generated_string = [
            re.sub(
                "<bos>|<eos>|<pad>",
                "",
                " ".join(list(map(self.vocab.__getitem__, item.tolist()))),
            )
            for i, item in enumerate(generated_sequence.detach().cpu())
        ]
        generated_string = [
            re.sub(r"\s+", " ", item.strip()) for item in generated_string
        ]
        return generated_string

说实话，beam search是这次作业最难的部分。
需要处理很多难点逻辑。

其中最难的就是和model之间的交互关系。

而且原本的 main.py 和 model.py 对于decode 的接口设计有问题，我们有 hidden_states =None, 这是一个重要的参数，但是decode 没有返回新的hidden_states, 这就导致我们在beam search的时候，无法使用上一个时刻的 hidden_states 来进行计算，会减慢速度。

我们重新设计了整个接口，结合多个网络代码和AI生成的代码，终于把正确的 beam search实现了出来。

In [ ]:
def beam_search(
    self,
    encoder_output: torch.Tensor,  # shape (batch_size, sequence_length, hidden_size) representing the encoder output
    source_mask: torch.Tensor,  # shape (batch_size, sequence_length) representing the input mask
) -> torch.Tensor:  # int ids, shape (batch_size, max_length)
    """
    perform beam search to get the generated sequence with the highest score
    """
    # 0. get necessary info
    batch_size, seq_length, hidden_size = encoder_output.shape
    num_beams = self.num_beams
    vocab_size = self.vocab_size
    max_length = self.max_length

    # 1. prepare the encoder output and source mask
    encoder_output = einops.repeat(
        encoder_output, "b s h -> (b n) s h", n=num_beams
    )
    source_mask = einops.repeat(source_mask, "b s -> (b n) s", n=num_beams)

    # 2. 初始化为(batch*beam, 1)格式, storing the top-`num_beams` generated sequence
    sequence = torch.full(
        (batch_size * num_beams, 1),
        self.bos_index,  # 填满这个 begin of sentence
        dtype=torch.long,
        device=self.device,
    )
    # 3. beam scores, corresponding scores at current timestep
    beam_scores = torch.zeros(batch_size, num_beams, device=self.device)
    beam_scores[:, 1:] = -1e9
    beam_scores = beam_scores.view(-1)

    # 4. a flag tensor indicating whether generation is done for current sentence
    finished = torch.zeros(batch_size, dtype=torch.bool, device=self.device)
    hypotheses = [
        BeamHypotheses(num_beams, self.length_penalty) for _ in range(batch_size)
    ]
    hidden_states = None  # 有点奇怪

    # Beam search loop
    for cur_len in range(1, max_length):
        # Get decoder output
        decoder_output, new_hidden_states = self.model.decode(
            encoder_outputs=encoder_output,
            source_mask=source_mask,
            target_input_ids=sequence,
            target_mask=torch.ones_like(sequence).to(self.device),
            hidden_states=hidden_states
        )

        # Get logits for the next token
        logits = decoder_output[:, -1, :]  # (batch_size * num_beams, vocab_size)
        log_probs = torch.log_softmax(logits, dim=-1)

        # Add log_probs to beam_scores
        next_scores = beam_scores.unsqueeze(-1) + log_probs  # (batch_size * num_beams, vocab_size)
        next_scores = next_scores.view(batch_size, num_beams * vocab_size)

        # Get top k scores and indices
        next_scores, next_indices = torch.topk(next_scores, num_beams, dim=1)
        next_beam_indices = next_indices // vocab_size
        next_token_indices = next_indices % vocab_size

        # Update beam_scores
        beam_scores = next_scores.view(-1)

        # Update sequence
        sequence = sequence[next_beam_indices.view(-1)]
        sequence = torch.cat(
            [sequence, next_token_indices.view(-1, 1)], dim=1
        )

        # Update hidden states
        if new_hidden_states is not None:
            hidden_states = new_hidden_states[:, next_beam_indices.view(-1), :]
            # hidden_states = new_hidden_states

        # Check for EOS tokens
        eos_in_beam = (next_token_indices == self.eos_index).any(dim=1)
        for i in range(batch_size):
            if eos_in_beam[i].any():
                beam_idx = i * num_beams
                for j in range(num_beams):
                    if beam_idx + j < next_token_indices.size(0) and next_token_indices[beam_idx + j] == self.eos_index:
                        hypotheses[i].add(
                            sequence[beam_idx + j].clone(),
                            beam_scores[beam_idx + j].item()
                        )
                if len(hypotheses[i]) >= num_beams:
                    finished[i] = True

        if finished.all():
            break

    # Get the best sequence from hypotheses
    best_sequences = []
    for i in range(batch_size):
        if len(hypotheses[i]) > 0:
            best_sequences.append(hypotheses[i].beams[-1][0])
        else:
            best_sequences.append(sequence[i * num_beams])

    # Pad sequences to max_length
    output_sequence = torch.full(
        (batch_size, max_length), self.eos_index, dtype=torch.long, device=self.device
    )
    for i, seq in enumerate(best_sequences):
        seq_len = min(len(seq), max_length)
        output_sequence[i, :seq_len] = seq[:seq_len]

    return output_sequence

## RWKV 实现

RWKV 是 目前较为先进的RNN模型，号称结合了 Attention 并行训练和长文本建模的能力和RNN高效推理无限上下文长度的能力。RWKV 本质上和LSTM一样都是RNN模型，只是其使用了不同的激活函数和门控机制以及其他的一些高级操作。

感觉RWKV的开源做的很好，在linux基金会下 https://rwkv.cn/news/read?id=15 ，看起来很有前景。

RWKV-LM 的开源代码很复杂，有cuda kernel c++啥的，很难改。通过和RWKV社区成员沟通交流，发现这个https://github.com/TorchRWKV/flash-linear-attention/tree/stable 实现比较优雅，用torch写，但是用triton编译。

我们安装一下。

```bash
pip install -U git+https://github.com/TorchRWKV/flash-linear-attention
```

看源码 https://github.com/TorchRWKV/flash-linear-attention/blob/stable/fla/layers/rwkv7.py



In [ ]:
# 引入 RWKV7Attention 模块
from fla.layers.rwkv7 import RWKV7Attention
rwkv = RWKV7Attention(mode="chunk", hidden_size=vector_size, head_dim=64)

## 运行效果

### 跑通前其他报错

![alt text](image-14.png)

这个错误来自 ELMoForManyLangs 库的 Highway 类的实现问题。看起来是 Python 3.12 的类型检查更严格了，导致 overrides 装饰器检查失败。我们需要修复 highway.py 文件中的问题。

![alt text](image-15.png)

这个错误是因为 外面的 util和里面的util重名了（我们刚才强行使用sys把m2scorer加入到解释器路径中）。
按照thu-cvml 命名规范，我们把外面的改成infra.py 而不是 util.py 避免冲突

/home/ycm/repos/coursework/THU-Coursework-Knowledge-Engineering/7.中文语法错误纠正的作业/main.py:122: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():

### 训练准确率

![alt text](image-16.png)

可以看到GRU模型的准确率随着epoch进行逐渐上升, 从 35 上升到了 65

![alt text](image-17.png)

RWKV 报错，由于时间不足，我们下次再跑。

beam search 可以运行，但是速度太慢了，我们把它从训练代码中去除了，否则需要跑很多小时才能跑出来

![alt text](image-18.png)

